In [0]:
from pyspark.sql.functions import count, sum, min, max, lit, size
from pyspark.sql.window import Window

In [0]:

df_conversations_silver = spark.read.table("slack_etl.silver.conversations")
df_users_silver = spark.read.table("slack_etl.silver.users")

df_message_attachments_silver = spark.read.table("slack_etl.silver.message_attachments")
df_message_reactions_silver = spark.read.table("slack_etl.silver.message_reactions")
df_message_threads_silver = spark.read.table("slack_etl.silver.message_threads")

In [0]:


df_message_attachments_gold_enriched = df_message_attachments_silver.join(df_conversations_silver, 'id', 'left').drop('purpose', 'topic','creator')
df_message_attachments_gold_file_stats = df_message_attachments_gold_enriched.groupBy('channel_name').agg(count('attachment_id').alias('attachment_count'), sum('size').alias('total_size_in_bytes'))

df_message_attachments_gold_file_stats.write.format('delta').mode('overwrite').saveAsTable('slack_etl.gold.messages_attachment_stats')

In [0]:
df_message_reactions_gold_enriched = df_message_reactions_silver.join(df_conversations_silver, 'id', 'left').drop('purpose', 'topic','creator')

df_message_reactions_gold_stats = df_message_reactions_gold_enriched.groupBy('channel_name', 'name').agg(count('*').alias('count'))

df_message_reactions_gold_stats.write.format('delta').mode('overwrite').saveAsTable('slack_etl.gold.messages_reaction_stats')

In [0]:
df_users_gold_location_stats = df_users_silver.groupBy('location').count().orderBy('count', ascending=False)
df_users_gold_department_stats = df_users_silver.groupBy('department').count().orderBy('count', ascending=False)
df_users_gold_department_stats.write.format('delta').mode('overwrite').saveAsTable('slack_etl.gold.users_department_stats')


df_users_gold_department_stats.write.mode('overwrite').saveAsTable('slack_etl.gold.users_department_stats')
df_users_gold_location_stats.write.mode('overwrite').saveAsTable('slack_etl.gold.users_location_stats')
df_users_gold_location_stats.write.format('delta').mode('overwrite').saveAsTable('slack_etl.gold.users_location_stats')

In [0]:
channel_count = spark.sql('SELECT COUNT(*) from slack_etl.silver.conversations').collect()[0][0]

df_conversation_gold_stats = (
    df_conversations_silver
    .withColumn('member_count', size('members'))
    .withColumn('channel_count', lit(channel_count))
).select('channel_name', 'member_count', 'channel_count')

df_conversation_gold_stats.write.format('delta').mode('overwrite').saveAsTable('slack_etl.gold.conversations_stats')

In [0]:
df_coversation_messages = df_message_threads_silver.join(df_conversations_silver, 'id', 'left').drop('purpose', 'topic','creator')

df_coversation_messages_enriched = df_coversation_messages.groupBy('channel_name').agg(count('client_msg_id').alias('message_count'))
df_coversation_messages_enriched.write.format('delta').mode('overwrite').saveAsTable('slack_etl.gold.conversation_messages')